In [1]:
import os
%pwd

'/Applications/AI/GAN/research'

In [2]:
os.chdir('../')
%pwd

'/Applications/AI/GAN'

In [3]:
# GAN Model Training: entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class GANTrainingConfig:
    root_dir: Path
    trained_generator_path: Path
    trained_discriminator_path: Path
    generator_model_path: Path
    discriminator_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_image_size: list
    params_learning_rate: float
    params_beta1: float
    params_lambda_cycle: int
    params_lambda_identity: float
    params_save_interval: int

# GAN Model Training: config manager
from src.GAN.constants import *
from src.GAN.utils.common import read_yaml, create_directories
import tensorflow as tf
import os

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_gan_training_config(self) -> GANTrainingConfig:
        training = self.config.gan_training
        prepare_gan_model = self.config.prepare_gan_model
        params = self.params
        
        # Look for data in standard data ingestion paths
        base_data_path = self.config.data_ingestion.unzip_dir
        
        # Try several possible paths where Monet images might be
        possible_paths = [
            os.path.join(base_data_path, "monet_jpg"),
            os.path.join(base_data_path, "gan-getting-started/monet_jpg"),
            os.path.join(base_data_path, "monet_tfrec"),
            base_data_path  # Fallback to the main unzip directory
        ]
        
        # Find the first path that exists
        training_data = None
        for path in possible_paths:
            if os.path.exists(path):
                training_data = path
                logger.info(f"Found training data at: {path}")
                break
        
        if training_data is None:
            logger.warning(f"Could not find training data in expected locations. Using {base_data_path}")
            training_data = base_data_path
        
        create_directories([Path(training.root_dir)])

        gan_training_config = GANTrainingConfig(
            root_dir=Path(training.root_dir),
            trained_generator_path=Path(training.trained_generator_path),
            trained_discriminator_path=Path(training.trained_discriminator_path),
            generator_model_path=Path(prepare_gan_model.generator_model_path),
            discriminator_model_path=Path(prepare_gan_model.discriminator_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE,
            params_beta1=params.BETA1,
            params_lambda_cycle=params.LAMBDA_CYCLE,
            params_lambda_identity=params.LAMBDA_IDENTITY,
            params_save_interval=params.SAVE_INTERVAL
        )

        return gan_training_config

# GAN Model Training: component
import os
import time
import tensorflow as tf
from src.GAN import logger
import matplotlib.pyplot as plt

class GANTraining:
    def __init__(self, config: GANTrainingConfig):
        self.config = config
    
    def load_models(self):
        """Load the generator and discriminator models"""
        logger.info("Loading generator and discriminator models")
        self.generator = tf.keras.models.load_model(self.config.generator_model_path)
        self.discriminator = tf.keras.models.load_model(self.config.discriminator_model_path)
        
        # Optimizers
        self.generator_optimizer = tf.keras.optimizers.Adam(
            learning_rate=self.config.params_learning_rate, 
            beta_1=self.config.params_beta1
        )
        self.discriminator_optimizer = tf.keras.optimizers.Adam(
            learning_rate=self.config.params_learning_rate, 
            beta_1=self.config.params_beta1
        )
    
    def load_dataset(self):
        """Load and prepare image datasets for GAN training"""
        logger.info("Preparing datasets")
        
        # Find correct data paths
        data_root = self.config.training_data.parent
        monet_dir = self.find_directory_with_images(data_root, "monet")
        photo_dir = self.find_directory_with_images(data_root, "photo")
        
        if not monet_dir or not photo_dir:
            raise ValueError(f"Could not find monet and photo directories in {data_root}")
        
        logger.info(f"Found Monet images in {monet_dir}")
        logger.info(f"Found photo images in {photo_dir}")
        
        # Load Monet paintings dataset
        monet_files = self.get_image_files(monet_dir)
        self.monet_dataset = self.create_dataset_from_files(
            monet_files, 
            self.config.params_batch_size, 
            self.config.params_image_size
        )
        
        # Load photo dataset
        photo_files = self.get_image_files(photo_dir)
        self.photo_dataset = self.create_dataset_from_files(
            photo_files,
            self.config.params_batch_size,
            self.config.params_image_size
        )
        
        # Create training datasets that cycle indefinitely
        self.monet_dataset = self.monet_dataset.repeat()
        self.photo_dataset = self.photo_dataset.repeat()
    
    def find_directory_with_images(self, root_dir, name_contains):
        """Find a directory that contains images and has 'name_contains' in its path"""
        for root, dirs, files in os.walk(root_dir):
            image_files = [f for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            if image_files and name_contains.lower() in root.lower():
                return root
        return None
    
    def get_image_files(self, directory):
        """Get all image files in a directory"""
        extensions = ['.jpg', '.jpeg', '.png']
        files = []
        for ext in extensions:
            files.extend([os.path.join(directory, f) for f in os.listdir(directory)
                         if f.lower().endswith(ext)])
        return files
    
    def create_dataset_from_files(self, file_paths, batch_size, image_size):
        """Create a dataset from image file paths"""
        dataset = tf.data.Dataset.from_tensor_slices(file_paths)
        dataset = dataset.map(
            lambda x: self.load_and_preprocess_image(x, image_size),
            num_parallel_calls=tf.data.AUTOTUNE
        )
        dataset = dataset.batch(batch_size)
        dataset = dataset.prefetch(tf.data.AUTOTUNE)
        return dataset
    
    def load_and_preprocess_image(self, file_path, image_size):
        """Load and preprocess an image"""
        img = tf.io.read_file(file_path)
        img = tf.io.decode_jpeg(img, channels=3)
        # Ensure we use exact dimensions from params
        img = tf.image.resize(img, (256, 256))  # Hard-code 256×256 to match models
        img = (tf.cast(img, tf.float32) / 127.5) - 1  # Normalize to [-1, 1]
        return img
    
    def discriminator_loss(self, real, generated):
        """Discriminator loss function"""
        real_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(
            tf.ones_like(real), real)
        generated_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(
            tf.zeros_like(generated), generated)
        return (real_loss + generated_loss) * 0.5
    
    def generator_loss(self, generated):
        """Generator loss function"""
        return tf.keras.losses.BinaryCrossentropy(from_logits=True)(
            tf.ones_like(generated), generated)
    
    @tf.function
    def train_step(self, real_photos, real_monet):
        """Single training step for image-to-image GAN"""
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            # Generate Monet-style image from photo
            fake_monet = self.generator(real_photos, training=True)
            
            # Resize fake images if dimensions don't match
            if fake_monet.shape[1:3] != real_monet.shape[1:3]:
                fake_monet = tf.image.resize(fake_monet, (256, 256))
            
            # Discriminator predictions
            real_monet_output = self.discriminator(real_monet, training=True)
            fake_monet_output = self.discriminator(fake_monet, training=True)
            
            # Calculate losses
            gen_loss = self.generator_loss(fake_monet_output)
            disc_loss = self.discriminator_loss(real_monet_output, fake_monet_output)
        
        # Calculate gradients
        gen_gradients = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        disc_gradients = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)
        
        # Apply gradients
        self.generator_optimizer.apply_gradients(zip(gen_gradients, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(disc_gradients, self.discriminator.trainable_variables))
        
        return gen_loss, disc_loss
    
    def save_sample_images(self, epoch, photos):
        """Generate and save sample images"""
        predictions = self.generator(photos, training=False)
        
        plt.figure(figsize=(12, 12))
        
        for i in range(min(4, len(predictions))):
            # Display original photo
            plt.subplot(4, 2, i*2 + 1)
            plt.title("Original")
            photo = (photos[i] + 1) * 0.5  # Denormalize
            plt.imshow(photo)
            plt.axis('off')
            
            # Display generated Monet-style image
            plt.subplot(4, 2, i*2 + 2)
            plt.title("Generated")
            prediction = (predictions[i] + 1) * 0.5  # Denormalize
            plt.imshow(prediction)
            plt.axis('off')
        
        plt.savefig(os.path.join(self.config.root_dir, f'epoch_{epoch}.png'))
        plt.close()
    
    def train(self):
        """Train the GAN"""
        logger.info(f"Starting GAN training for {self.config.params_epochs} epochs")
        
        # Get sample photos for visualization
        sample_photos = next(iter(self.photo_dataset.take(1)))
        
        # Set a reasonable number of steps per epoch
        # For a small dataset like Monet paintings, 100 steps is reasonable
        steps_per_epoch = 100
        
        # Create iterators
        monet_iter = iter(self.monet_dataset)
        photo_iter = iter(self.photo_dataset)
        
        for epoch in range(self.config.params_epochs):
            start = time.time()
            logger.info(f"Epoch {epoch+1}/{self.config.params_epochs}")
            
            # Training loop
            for step in range(steps_per_epoch):
                real_monet = next(monet_iter)
                real_photos = next(photo_iter)
                
                gen_loss, disc_loss = self.train_step(real_photos, real_monet)
                
                if step % 10 == 0:
                    logger.info(f"Step {step}/{steps_per_epoch} - Gen loss: {gen_loss:.4f}, Disc loss: {disc_loss:.4f}")
            
            # Generate sample images
            self.save_sample_images(epoch+1, sample_photos)
            
            # Save models periodically
            if (epoch + 1) % self.config.params_save_interval == 0:
                self.save_models(epoch + 1)
            
            logger.info(f"Time for epoch {epoch+1}: {time.time()-start:.2f} sec")
        
        # Save final models
        self.save_models("final")
        logger.info("GAN training completed")
    
    def save_models(self, epoch):
        """Save the generator and discriminator models"""
        generator_path = os.path.join(self.config.root_dir, f"generator_epoch_{epoch}.h5")
        discriminator_path = os.path.join(self.config.root_dir, f"discriminator_epoch_{epoch}.h5")
        
        self.generator.save(generator_path)
        self.discriminator.save(discriminator_path)
        
        # Save final models with the original names
        if epoch == "final":
            self.generator.save(self.config.trained_generator_path)
            self.discriminator.save(self.config.trained_discriminator_path)
            logger.info(f"Final models saved at {self.config.trained_generator_path} and {self.config.trained_discriminator_path}")

# GAN Model Training: pipeline
try:
    config = ConfigurationManager()
    gan_training_config = config.get_gan_training_config()
    gan_training = GANTraining(config=gan_training_config)
    gan_training.load_models()
    gan_training.load_dataset()
    gan_training.train()
except Exception as e:
    raise e

/Applications/AI/GAN/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[2025-03-20 10:20:12,271: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-20 10:20:12,274: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-20 10:20:12,275: INFO: common: created directory at: artifacts]
[2025-03-20 10:20:12,276: INFO: 2118690621: Found training data at: artifacts/data_ingestion/gan-getting-started/monet_jpg]
[2025-03-20 10:20:12,277: INFO: common: created directory at: artifacts/gan_training]
[2025-03-20 10:20:12,277: INFO: 2118690621: Loading generator and discriminator models]
[2025-03-20 10:20:12,720: WARNING: optimizer: At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.]
[2025-03-20 10:20:12,722: WARNING: optimizer: At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam

/Applications/AI/GAN/.venv/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


[2025-03-20 10:54:08,847: INFO: 2118690621: Final models saved at artifacts/gan_training/generator_final.h5 and artifacts/gan_training/discriminator_final.h5]
[2025-03-20 10:54:08,849: INFO: 2118690621: GAN training completed]
